In [6]:
!pip install kagglehub pandas numpy scikit-learn gradio requests
!pip install --upgrade kagglehub

import kagglehub
import pandas as pd
import numpy as np
import ast
import pickle
import requests
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import ast
import pickle
import requests
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Download the dataset using kagglehub
print("Downloading TMDB movie dataset...")
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")
print(f"Dataset downloaded to: {path}")

# Load the CSV files
movies = pd.read_csv(f"{path}/tmdb_5000_movies.csv")
credits = pd.read_csv(f"{path}/tmdb_5000_credits.csv")

print(f"Movies dataset shape: {movies.shape}")
print(f"Credits dataset shape: {credits.shape}")

# Display first few rows
print("\nFirst 2 rows of movies dataset:")
movies.head(2)


Dataset downloaded to: /kaggle/input/tmdb-movie-metadata
Movies dataset shape: (4803, 20)
Credits dataset shape: (4803, 4)

First 2 rows of movies dataset:


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [ ]:
# Display credits dataset
print("First 2 rows of credits dataset:")
credits.head(2)


First 2 rows of credits dataset:


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
# Merge movies and credits datasets
movies = movies.merge(credits, on='title')
print(f"Merged dataset shape: {movies.shape}")

# Select relevant columns
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
print("\nSelected columns:")
movies.head()


Merged dataset shape: (4809, 23)

Selected columns:


,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
# Define helper functions for data preprocessing

def convert(text):
    """Convert JSON string to list of names"""
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

def convert3(text):
    """Convert JSON string to list of first 3 names"""
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter += 1
    return L

def fetch_director(text):
    """Extract director names from crew data"""
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

def collapse(L):
    """Remove spaces from list items"""
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

print("Helper functions defined successfully!")


Helper functions defined successfully!


In [ ]:
# Remove null values
movies.dropna(inplace=True)
print(f"Dataset shape after removing null values: {movies.shape}")

# Process genres
movies['genres'] = movies['genres'].apply(convert)
print("Genres processed")

# Process keywords
movies['keywords'] = movies['keywords'].apply(convert)
print("Keywords processed")

# Process cast (top 3 actors)
movies['cast'] = movies['cast'].apply(convert)
movies['cast'] = movies['cast'].apply(lambda x: x[0:3])
print("Cast processed")

# Process crew (directors only)
movies['crew'] = movies['crew'].apply(fetch_director)
print("Crew processed")

# Display sample data
print("\nSample processed data:")
movies.sample(3)


Dataset shape after removing null values: (4806, 7)
Genres processed
Keywords processed
Cast processed
Crew processed

Sample processed data:


,movie_id,title,overview,genres,keywords,cast,crew
1385,13920,Radio,"High school football coach, Harold Jones befri...",[Drama],"[biography, friendship, sport]","[Cuba Gooding Jr., Ed Harris, Alfre Woodard]",[Mike Tollin]
558,35,The Simpsons Movie,After Homer accidentally pollutes the town's w...,"[Animation, Comedy, Family]","[father son relationship, lake, springfield, t...","[Dan Castellaneta, Julie Kavner, Nancy Cartwri...",[David Silverman]
3438,26665,Aberdeen,"Kaisa is a Scot, a successful London lawyer, w...","[Drama, Comedy, Foreign]","[alcohol, cocaine, daughter, road trip, indepe...","[Stellan Skarsgård, Lena Headey, Ian Hart]",[Hans Petter Moland]


In [ ]:
# Remove spaces from all text data
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

# Convert overview to list of words
movies['overview'] = movies['overview'].apply(lambda x: x.split())

# Create tags by combining all features
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

# Create final dataset
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

print(f"Final dataset shape: {new.shape}")
print("\nFinal processed data:")
new.head()


Final dataset shape: (4806, 3)

Final processed data:


,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [ ]:
# Create count vectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()
print(f"Vector shape: {vector.shape}")

# Calculate cosine similarity
similarity = cosine_similarity(vector)
print(f"Similarity matrix shape: {similarity.shape}")

print("\nModel created successfully!")


Vector shape: (4806, 5000)
Similarity matrix shape: (4806, 4806)

Model created successfully!


In [ ]:
# Basic recommendation function
def recommend(movie):
    """Recommend movies based on similarity"""
    try:
        index = new[new['title'] == movie].index[0]
        distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
        recommended_movies = []

        print(f"\nRecommendations for '{movie}':")
        print("-" * 50)

        for i in distances[1:6]:
            movie_title = new.iloc[i[0]].title
            similarity_score = round(i[1] * 100, 2)
            print(f"{len(recommended_movies)+1}. {movie_title} (Similarity: {similarity_score}%)")
            recommended_movies.append(movie_title)

        return recommended_movies
    except IndexError:
        print(f"Movie '{movie}' not found in the database.")
        return []

# Test the recommendation function
test_recommendations = recommend('Avatar')



Recommendations for 'Avatar':
--------------------------------------------------
1. Titan A.E. (Similarity: 25.37%)
2. Small Soldiers (Similarity: 25.11%)
3. Ender's Game (Similarity: 24.42%)
4. Aliens vs Predator: Requiem (Similarity: 24.26%)
5. Independence Day (Similarity: 24.17%)


In [ ]:
# Test recommendations for different movies
test_movies = ['The Dark Knight', 'Titanic', 'Avatar', 'Iron Man', 'Gandhi']

for movie in test_movies:
    if movie in new['title'].values:
        recommend(movie)
        print("\n")
    else:
        print(f"'{movie}' not found in dataset")



Recommendations for 'The Dark Knight':
--------------------------------------------------
1. The Dark Knight Rises (Similarity: 42.39%)
2. Batman Begins (Similarity: 39.39%)
3. Batman Returns (Similarity: 32.16%)
4. Batman Forever (Similarity: 28.79%)
5. Batman & Robin (Similarity: 26.79%)



Recommendations for 'Titanic':
--------------------------------------------------
1. Captain Phillips (Similarity: 28.18%)
2. The Notebook (Similarity: 26.55%)
3. In the Heart of the Sea (Similarity: 25.68%)
4. Ghost Ship (Similarity: 25.57%)
5. Poseidon (Similarity: 24.33%)



Recommendations for 'Avatar':
--------------------------------------------------
1. Titan A.E. (Similarity: 25.37%)
2. Small Soldiers (Similarity: 25.11%)
3. Ender's Game (Similarity: 24.42%)
4. Aliens vs Predator: Requiem (Similarity: 24.26%)
5. Independence Day (Similarity: 24.17%)



Recommendations for 'Iron Man':
--------------------------------------------------
1. Iron Man 2 (Similarity: 46.85%)
2. Iron Man 3 (Simil

In [7]:
# Download the dataset using kagglehub
print("Downloading TMDB movie dataset...")
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")
print(f"Dataset downloaded to: {path}")

# Load the CSV files
movies = pd.read_csv(f"{path}/tmdb_5000_movies.csv")
credits = pd.read_csv(f"{path}/tmdb_5000_credits.csv")

print(f"Movies dataset shape: {movies.shape}")
print(f"Credits dataset shape: {credits.shape}")

# Merge datasets
movies = movies.merge(credits, on='title')
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

# Define helper functions
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

# Process data
movies.dropna(inplace=True)
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert)
movies['cast'] = movies['cast'].apply(lambda x: x[0:3])
movies['crew'] = movies['crew'].apply(fetch_director)

movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

movies['overview'] = movies['overview'].apply(lambda x: x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

print(f"Final dataset shape: {new.shape}")
print("Data preprocessing completed!")
new.head()


Dataset downloaded to: /kaggle/input/tmdb-movie-metadata
Movies dataset shape: (4803, 20)
Credits dataset shape: (4803, 4)
Final dataset shape: (4806, 3)
Data preprocessing completed!


,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [8]:
# Create count vectorizer and similarity matrix
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()
similarity = cosine_similarity(vector)

print(f"Vector shape: {vector.shape}")
print(f"Similarity matrix shape: {similarity.shape}")
print("Model created successfully!")


Vector shape: (4806, 5000)
Similarity matrix shape: (4806, 4806)
Model created successfully!


In [10]:
# Enhanced poster fetching function
def fetch_poster(movie_id):
    """Fetch movie poster from TMDB API"""
    try:
        url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US"
        response = requests.get(url)
        data = response.json()

        if 'poster_path' in data and data['poster_path']:
            poster_url = "https://image.tmdb.org/t/p/w500" + data['poster_path']
            return poster_url
        else:
            return "https://via.placeholder.com/500x750/1f2937/ffffff?text=No+Image+Available"
    except:
        return "https://via.placeholder.com/500x750/1f2937/ffffff?text=No+Image+Available"

# Get movie details
def get_movie_details(movie_title):
    """Get additional movie details"""
    try:
        movie_row = new[new['title'] == movie_title].iloc[0]
        movie_id = movie_row['movie_id']

        # Fetch additional details from TMDB API
        url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US"
        response = requests.get(url)
        data = response.json()

        details = {
            'title': movie_title,
            'year': data.get('release_date', 'N/A')[:4] if data.get('release_date') else 'N/A',
            'rating': data.get('vote_average', 'N/A'),
            'genres': ', '.join([genre['name'] for genre in data.get('genres', [])]),
            'overview': data.get('overview', 'No overview available'),
            'runtime': f"{data.get('runtime', 'N/A')} min" if data.get('runtime') else 'N/A',
            'poster_url': fetch_poster(movie_id)
        }
        return details
    except:
        return {
            'title': movie_title,
            'year': 'N/A',
            'rating': 'N/A',
            'genres': 'N/A',
            'overview': 'Details not available',
            'runtime': 'N/A',
            'poster_url': "https://via.placeholder.com/500x750/1f2937/ffffff?text=No+Image+Available"
        }

# Enhanced recommendation function
def enhanced_recommend(movie_name):
    """Enhanced recommendation with movie details and posters"""
    if not movie_name:
        return None, "Please select a movie first!", ""

    if movie_name not in new['title'].values:
        return None, f"Movie '{movie_name}' not found in database.", ""

    try:
        # Get selected movie details
        selected_movie_details = get_movie_details(movie_name)

        # Get recommendations
        index = new[new['title'] == movie_name].index[0]
        distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])

        # Create HTML for selected movie
        poster_url = selected_movie_details['poster_url']
        title = selected_movie_details['title']
        rating = selected_movie_details['rating']
        year = selected_movie_details['year']
        runtime = selected_movie_details['runtime']
        genres = selected_movie_details['genres']
        overview = selected_movie_details['overview'][:200]

        selected_info = f"""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 20px; padding: 25px; color: white; margin-bottom: 20px;">
            <div style="display: flex; align-items: center; gap: 20px; flex-wrap: wrap;">
                <img src="{poster_url}" style="width: 120px; height: 180px; object-fit: cover; border-radius: 15px; box-shadow: 0 8px 16px rgba(0,0,0,0.3);" alt="{movie_name}">
                <div style="flex: 1; min-width: 300px;">
                    <h2 style="margin: 0 0 10px 0; font-size: 28px; font-weight: bold;">{title}</h2>
                    <p style="margin: 5px 0; font-size: 16px;">⭐ {rating}/10 | {year} | {runtime}</p>
                    <p style="margin: 10px 0; font-size: 14px; opacity: 0.9;">{genres}</p>
                    <p style="margin: 15px 0 0 0; font-size: 14px; line-height: 1.5; opacity: 0.95;">{overview}...</p>
                </div>
            </div>
        </div>
        """

        # Create recommendations HTML
        recommendations_html = f'<div style="margin-top: 20px;"><h3 style="color: #1e293b; margin-bottom: 20px; text-align: center;">🎬 Top 5 Recommendations for "{movie_name}"</h3><div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 20px; padding: 20px; background: #f8fafc; border-radius: 15px;">'

        for i in distances[1:6]:
            movie_title = new.iloc[i[0]].title
            similarity_score = round(i[1] * 100, 2)
            movie_details = get_movie_details(movie_title)

            rec_poster = movie_details['poster_url']
            rec_rating = movie_details['rating']
            rec_year = movie_details['year']
            rec_genres = movie_details['genres']

            recommendations_html += f"""
            <div style="background: white; border-radius: 15px; padding: 15px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); transition: transform 0.3s ease; border: 1px solid #e2e8f0;">
                <img src="{rec_poster}" style="width: 100%; height: 250px; object-fit: cover; border-radius: 10px; margin-bottom: 10px;" alt="{movie_title}">
                <h4 style="margin: 10px 0 5px 0; color: #1e293b; font-size: 14px; font-weight: bold; height: 40px; display: flex; align-items: center;">{movie_title}</h4>
                <p style="margin: 5px 0; color: #64748b; font-size: 12px;">⭐ {rec_rating}/10 | {rec_year}</p>
                <p style="margin: 5px 0; color: #10b981; font-size: 12px; font-weight: bold;">Similarity: {similarity_score}%</p>
                <p style="margin: 5px 0; color: #6b7280; font-size: 11px; height: 30px; overflow: hidden;">{rec_genres}</p>
            </div>
            """

        recommendations_html += '</div></div>'

        final_output = selected_info + recommendations_html

        return selected_movie_details['poster_url'], final_output, f"✅ Found 5 similar movies for '{movie_name}'"

    except Exception as e:
        return None, f"Error generating recommendations: {str(e)}", "❌ Error occurred"

# Search function for movies
def search_movies(query):
    """Search movies by title"""
    if not query:
        return "Enter a movie title to search..."

    matches = new[new['title'].str.contains(query, case=False, na=False)]['title'].tolist()

    if not matches:
        return f"No movies found for '{query}'"

    # Display search results
    results_html = f'<div style="margin-top: 20px;"><h4 style="color: #1e293b; margin-bottom: 15px;">🔍 Found {len(matches)} movies for "{query}":</h4><div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(250px, 1fr)); gap: 15px;">'

    for movie in matches[:12]:  # Show max 12 results
        movie_details = get_movie_details(movie)
        movie_poster = movie_details['poster_url']
        movie_rating = movie_details['rating']
        movie_year = movie_details['year']
        movie_genres = movie_details['genres']

        results_html += f"""
        <div style="background: white; border: 1px solid #e2e8f0; border-radius: 10px; padding: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
            <img src="{movie_poster}" style="width: 100%; height: 200px; object-fit: cover; border-radius: 8px; margin-bottom: 10px;" alt="{movie}">
            <h5 style="margin: 0 0 5px 0; color: #1e293b; font-size: 14px; font-weight: bold;">{movie}</h5>
            <p style="margin: 2px 0; color: #64748b; font-size: 12px;">⭐ {movie_rating}/10 | {movie_year}</p>
            <p style="margin: 5px 0 0 0; color: #6b7280; font-size: 11px;">{movie_genres}</p>
        </div>
        """

    results_html += '</div></div>'
    return results_html

# Create the main interface
def create_movie_interface():
    # Get list of movies for dropdown
    movie_list = sorted(new['title'].tolist())

    # Custom CSS
    css = """
    .gradio-container {
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif !important;
        max-width: 1200px !important;
        margin: 0 auto !important;
    }
    .gr-button {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        border: none !important;
        color: white !important;
        font-weight: bold !important;
        border-radius: 10px !important;
        padding: 12px 24px !important;
    }
    """

    with gr.Blocks(css=css, title="🎬 Movie Recommendation System") as interface:

        # Header
        header_html = """
        <div style="text-align: center; padding: 30px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 20px; margin-bottom: 30px; box-shadow: 0 10px 25px rgba(0,0,0,0.1);">
            <h1 style="color: white; font-size: 48px; margin: 0; font-weight: bold; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
                🎬 Movie Recommendation System
            </h1>
            <p style="color: white; font-size: 18px; margin: 10px 0 0 0; opacity: 0.9;">
                Discover your next favorite movie with AI-powered recommendations
            </p>
        </div>
        """
        gr.HTML(header_html)

        with gr.Tab("🎯 Get Recommendations"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.HTML("<h3 style='color: #1e293b; margin-bottom: 15px;'>📽️ Select a Movie</h3>")

                    # Using basic Dropdown without unsupported parameters
                    movie_dropdown = gr.Dropdown(
                        choices=movie_list,
                        label="Choose from 5000+ movies",
                        value="Avatar"
                    )

                    recommend_btn = gr.Button(
                        "🎯 Get Recommendations",
                        variant="primary"
                    )

                    # Status display
                    status_display = gr.HTML()

                    # Database stats
                    stats_html = f"""
                    <div style="background: #f1f5f9; padding: 20px; border-radius: 15px; margin-top: 20px; border: 1px solid #e2e8f0;">
                        <h4 style="color: #334155; margin: 0 0 15px 0; text-align: center;">📊 Database Stats</h4>
                        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px; text-align: center;">
                            <div>
                                <p style="margin: 5px 0; color: #1e293b; font-size: 24px; font-weight: bold;">{len(new):,}</p>
                                <p style="margin: 0; color: #64748b; font-size: 12px;">Total Movies</p>
                            </div>
                            <div>
                                <p style="margin: 5px 0; color: #1e293b; font-size: 24px; font-weight: bold;">{vector.shape[1]:,}</p>
                                <p style="margin: 0; color: #64748b; font-size: 12px;">Features</p>
                            </div>
                        </div>
                    </div>
                    """
                    gr.HTML(stats_html)

                with gr.Column(scale=1):
                    selected_poster = gr.Image(
                        label="Selected Movie Poster",
                        height=400
                    )

            # Results display
            with gr.Row():
                recommendations_output = gr.HTML(label="Movie Details & Recommendations")

        with gr.Tab("🔍 Search Movies"):
            gr.HTML("<h3 style='color: #1e293b; margin-bottom: 20px;'>🎯 Search Movie Database</h3>")

            with gr.Row():
                search_input = gr.Textbox(
                    label="Search Movies",
                    placeholder="Enter movie title to search...",
                    lines=1
                )
                search_btn = gr.Button("🔍 Search", variant="secondary")

            search_results = gr.HTML()

        with gr.Tab("📊 About System"):
            about_html = f"""
            <div style="padding: 20px;">
                <h3 style="color: #1e293b; margin-bottom: 25px;">🤖 How the Recommendation System Works</h3>

                <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 25px; margin-bottom: 30px;">
                    <div style="background: linear-gradient(135deg, #3b82f6, #1e40af); color: white; padding: 25px; border-radius: 15px; text-align: center;">
                        <h4 style="margin: 0 0 15px 0; font-size: 20px;">📚 Dataset</h4>
                        <p style="font-size: 36px; font-weight: bold; margin: 10px 0;">{len(new):,}</p>
                        <p style="margin: 0; opacity: 0.9;">Movies Available</p>
                    </div>

                    <div style="background: linear-gradient(135deg, #10b981, #047857); color: white; padding: 25px; border-radius: 15px; text-align: center;">
                        <h4 style="margin: 0 0 15px 0; font-size: 20px;">🔍 Features</h4>
                        <p style="font-size: 36px; font-weight: bold; margin: 10px 0;">{vector.shape[1]:,}</p>
                        <p style="margin: 0; opacity: 0.9;">Unique Features</p>
                    </div>

                    <div style="background: linear-gradient(135deg, #8b5cf6, #7c3aed); color: white; padding: 25px; border-radius: 15px; text-align: center;">
                        <h4 style="margin: 0 0 15px 0; font-size: 20px;">⚡ Algorithm</h4>
                        <p style="font-size: 20px; font-weight: bold; margin: 10px 0;">Cosine Similarity</p>
                        <p style="margin: 0; opacity: 0.9;">Content-Based Filtering</p>
                    </div>
                </div>

                <div style="background: white; border: 2px solid #e2e8f0; border-radius: 15px; padding: 25px;">
                    <h4 style="color: #1e293b; margin: 0 0 20px 0;">🛠️ Process Flow</h4>
                    <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 20px;">
                        <div style="text-align: center; padding: 15px;">
                            <div style="background: #ddd6fe; width: 60px; height: 60px; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin: 0 auto 15px auto; font-size: 24px;">📝</div>
                            <h5 style="color: #1e293b; margin: 0 0 10px 0;">Data Processing</h5>
                            <p style="color: #64748b; font-size: 14px; line-height: 1.4;">Extract and clean movie metadata including genres, cast, crew, and keywords</p>
                        </div>
                        <div style="text-align: center; padding: 15px;">
                            <div style="background: #bfdbfe; width: 60px; height: 60px; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin: 0 auto 15px auto; font-size: 24px;">🔢</div>
                            <h5 style="color: #1e293b; margin: 0 0 10px 0;">Vectorization</h5>
                            <p style="color: #64748b; font-size: 14px; line-height: 1.4;">Convert text features into numerical vectors using CountVectorizer</p>
                        </div>
                        <div style="text-align: center; padding: 15px;">
                            <div style="background: #bbf7d0; width: 60px; height: 60px; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin: 0 auto 15px auto; font-size: 24px;">📐</div>
                            <h5 style="color: #1e293b; margin: 0 0 10px 0;">Similarity</h5>
                            <p style="color: #64748b; font-size: 14px; line-height: 1.4;">Calculate cosine similarity between all movie pairs</p>
                        </div>
                        <div style="text-align: center; padding: 15px;">
                            <div style="background: #fed7aa; width: 60px; height: 60px; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin: 0 auto 15px auto; font-size: 24px;">🎯</div>
                            <h5 style="color: #1e293b; margin: 0 0 10px 0;">Recommendations</h5>
                            <p style="color: #64748b; font-size: 14px; line-height: 1.4;">Return top 5 most similar movies based on content</p>
                        </div>
                    </div>
                </div>

                <div style="background: #f8fafc; border-radius: 10px; padding: 20px; margin-top: 20px; border-left: 4px solid #3b82f6;">
                    <h5 style="color: #1e293b; margin: 0 0 10px 0;">💡 Features Used for Recommendations:</h5>
                    <ul style="color: #64748b; margin: 0; padding-left: 20px;">
                        <li>Movie genres (Action, Drama, Comedy, etc.)</li>
                        <li>Plot keywords and themes</li>
                        <li>Main cast members (top 3 actors)</li>
                        <li>Director information</li>
                        <li>Movie overview and description</li>
                    </ul>
                </div>
            </div>
            """
            gr.HTML(about_html)

        # Event handlers
        recommend_btn.click(
            enhanced_recommend,
            inputs=[movie_dropdown],
            outputs=[selected_poster, recommendations_output, status_display]
        )

        search_btn.click(
            search_movies,
            inputs=[search_input],
            outputs=[search_results]
        )

        # Auto-update when dropdown changes
        movie_dropdown.change(
            enhanced_recommend,
            inputs=[movie_dropdown],
            outputs=[selected_poster, recommendations_output, status_display]
        )

        # Load initial recommendation
        interface.load(
            lambda: enhanced_recommend("Avatar"),
            outputs=[selected_poster, recommendations_output, status_display]
        )

    return interface

# Launch the interface
print("🚀 Launching Enhanced Movie Recommendation System...")
app = create_movie_interface()
app.launch(
    share=True,
    show_error=True,
    height=800
)


🚀 Launching Enhanced Movie Recommendation System...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b4d0f6521c109a87b2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
